In [ ]:
import math
import random

import gymnasium as gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

import matplotlib.pyplot as plt

In [ ]:
def set_global_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True        
    torch.backends.cudnn.benchmark = False

In [ ]:
set_global_seed(1)

In [ ]:
# use CUDA
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

### Create the environment

In [ ]:
# Create the environment and use multiprocessing (or not) to run multiple env in parallel when creating episodes for a policy

from stable_baselines3.common.vec_env import DummyVecEnv
    
num_envs = 16
env_name = "MountainCarContinuous-v0"

def make_env():
    def _thunk():
        env = gym.make(env_name)
        env._max_episode_steps = 1000
        return env

    return _thunk

envs = [make_env() for i in range(num_envs)]
envs = DummyVecEnv(envs)

env = gym.make(env_name)

### Neural Network


In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)
        

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [ ]:
from IPython.display import clear_output

def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20, 5))
    plt.subplot(131)    # 1 row and 3 column, chose the first case
    plt.title(f"frame {frame_idx}. reward: {rewards[-1]}")
    plt.plot(rewards)   # use the index as x
    plt.show()

def test_env(vis=False):
    # The single environment is a raw gymnasium env -> .reset() returns 2 values in a tuple
    state, _ = env.reset()
    
    if vis:
        env.render()
    
    done = False
    total_reward = 0

    while not done:
        state_tensor = (torch
                        .from_numpy(state)
                        .unsqueeze(0)
                        .to(device))
        
        dist, _ = model(state_tensor)   # the critic part of the model also return the sate value estimation

        # The single environment is a raw gymnasium env -> return 5 values in a tuple
        next_state, reward, terminated, truncated, _ = env.step(dist.sample().cpu().numpy()[0])
        done = terminated or truncated

        state = next_state
        total_reward += reward

        if vis:
            env.render()

    return total_reward


### GAE (Generalized Advatage Estimation)

In [ ]:
def compute_gae(next_value, rewards, masks, values, gamma=0.9999, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

### Proximal Policy Optimization Algorithm

In [ ]:
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage):
    batch_size = states.size(0)
    for _ in range(batch_size // mini_batch_size):
        rand_ids = np.random.randint(0, batch_size, mini_batch_size)
        yield states[rand_ids, :], actions[rand_ids, :], log_probs[rand_ids, :], returns[rand_ids, :], advantage[rand_ids, :]
        
        

def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, clip_param=0.2):
    for _ in range(ppo_epochs):
        for state, action, old_log_probs, return_, advantage in ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantages):
            dist, value = model(state)                      # using current policy and critic network
            entropy = dist.entropy().mean()                 # compute entropy to force exploration in the loss function
            new_log_probs = dist.log_prob(action)

            ratio = (new_log_probs - old_log_probs).exp()
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * advantage

            actor_loss  = - torch.min(surr1, surr2).mean()  # we take the mean over the minibatch to approximate the expected value of the surrogate loss
            critic_loss = (return_ - value).pow(2).mean()   # MSE ovet the minibatch

            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


In [ ]:
num_inputs  = envs.observation_space.shape[0]
num_outputs = envs.action_space.shape[0]

#Hyper params:
hidden_size      = 256
lr               = 3e-4
num_steps        = 2048
mini_batch_size  = 64
ppo_epochs       = 10
threshold_reward = 100

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
max_frames = 200000
frame_idx = 0
test_rewards = []

In [ ]:
import pandas as pd

state = envs.reset()
early_stop = False
total_steps = 0
csv_dict = {"num_step": [], "avg_reward": []}

while frame_idx < max_frames and not early_stop:

    log_probs = []
    values    = []
    states    = []
    actions   = []
    rewards   = []
    masks     = []
    entropy = 0

    for _ in range(num_steps):
        # ====== gather the data for the current policy ======
        state = torch.FloatTensor(state).to(device)
        dist, value = model(state)

        action = dist.sample()
        next_state, reward, done, _ = envs.step(action.cpu().numpy())

        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        states.append(state)
        actions.append(action)
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % 1000 == 0:
            test_reward = np.mean([test_env() for _ in range(10)])
            test_rewards.append(test_reward)
            plot(frame_idx, test_rewards)
            # save for the csv
            csv_dict["num_step"].append(total_steps)
            csv_dict["avg_reward"].append(test_reward)
            if test_reward > threshold_reward: early_stop = True
        #----------------------------------------------------
            

    # ====== Prepare the data for the PPO update and run the update ====== 
    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)

    returns   = torch.cat(returns).detach()
    log_probs = torch.cat(log_probs).detach()
    values    = torch.cat(values).detach()
    states    = torch.cat(states)
    actions   = torch.cat(actions)
    advantage = returns - values
    
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantage)

# save the csv file
csv_df = pd.DataFrame.from_dict(csv_dict)
csv_df.to_csv("ppo_training_rewards_mc_c.csv", index=False)

In [ ]:
test_rewards = [test_env() for _ in range(10)]
print(test_rewards.mean())
print(test_rewards.std())

In [ ]:
# Save the trained model’s parameters
answer = input("Are you sure you want to save the current model ?")
save_path = f"{env_name}.pth"

if answer=="yes":
    torch.save(model.state_dict(), save_path)
    print(f"Model parameters saved to {save_path}")

In [ ]:
# Create a fresh instance of the network and load the weights
loaded_model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
loaded_model.load_state_dict(torch.load(save_path, map_location=device))
loaded_model.eval()
print("Model loaded and set to eval mode")

# 3) Run one episode with rendering
render_env = gym.make(env_name, render_mode="human")
state, _ = render_env.reset()
done = False
total_reward = 0.0

while not done:
    render_env.render()
    # prepare state tensor
    state_tensor = torch.from_numpy(state).unsqueeze(0).to(device)
    with torch.no_grad():
        dist, _ = loaded_model(state_tensor)
    action = dist.mean.cpu().numpy()[0]  # deterministic for rendering
    
    # step the env
    next_state, reward, terminated, truncated, _ = render_env.step(action)
    done = terminated or truncated
    
    total_reward += reward
    state = next_state

print(f"Episode finished, total reward: {total_reward:.2f}")
render_env.close()

### Record a video

In [ ]:
from gym.wrappers import RecordVideo
from datetime import datetime
import glob
from IPython.display import HTML, display

# 1) Build a timestamp string
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

# 2) Wrap your environment, embedding the timestamp in the video name
video_env = RecordVideo(
    gym.make(env_name, render_mode="rgb_array"),
    video_folder="./videos",
    # This prefix (plus episode number) will ensure uniqueness
    name_prefix=f"{env_name}-{timestamp}",
    episode_trigger=lambda idx: True
)

# 3) Run one episode (no inline render calls)
state, _ = video_env.reset()
done = False
total_reward = 0.0

while not done:
    state_tensor = torch.from_numpy(state).unsqueeze(0).to(device)
    with torch.no_grad():
        dist, _ = loaded_model(state_tensor)
    action = dist.mean.cpu().numpy()[0]

    next_state, reward, terminated, truncated, _ = video_env.step(action)
    done = terminated or truncated
    total_reward += reward
    state = next_state

video_env.close()
print(f"Recorded episode, total reward = {total_reward:.2f}")
